<a href="https://colab.research.google.com/github/clarissa-souza/Desafio-rox/blob/main/Analise_Rox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cenário: Por falta de maiores detalhes, presumo que o cliente tem um datalake na plataforma GCP onde ele faz o upload de seus arquivos para pasta Dados Originais. Eu vou ler esse arquivo do datalake e fazer as análises e alterações que forem necessárias e, por fim, gravar o novo arquivo na pasta Dados Tratados.
Acredito que, pelas colunas das tabelas, essas tabelas sejam apenas uma view de uma parte do banco do ERP do cliente.


Análise: Consiste em identificar se existe e quais são as Primary Key e Foreing Key e o relacionamento entre as tabelas. Verificar se as colunas estão devidamente formatadas.

# Biblioteca

In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=50c47f24ff52f336aaad30fd14192e3bfd0dd771c83455a830ea67234677a6b9
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
# Importando as bibliotecas
import json
from datetime import datetime, date
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from functools import reduce
from pyspark.sql.types import StringType, IntegerType
from datetime import datetime, date


# Importando para o google cloud
from google.cloud import storage
import os

# Importando biblioteca para acessar a Service account que está no drive
from google.colab import drive

# Acesso ao bucket do cliente

In [ ]:
# Montando o acesso ao drive para acessar a Service account
drive.mount ('/content/drive')

Mounted at /content/drive


In [ ]:
# Configurando o caminho da Service account
serviceAccount='/content/drive/MyDrive/Rox/leega-estudos-6d979ec7a228.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=serviceAccount

# Instanciar o client
client=storage.Client()

# Receber o nome do bucket na variavel bucket
bucket=client.get_bucket('bucket-rox')

# Configurando o SparkSession

In [ ]:
#configurar a sparksession
spark = (SparkSession.builder
          .master('local') # quero gerar apenas um arquivo quando gravar o dado tratado no bucket então só vou usar o core local
          .appName('rox') # o nome do app
          .config('spark.ui.port','4050') # a porta onde vou rodar
          .config('spark.jars','https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop2-latest.jar')
          .getOrCreate()
          )

# Tabela Person: BusinessEntityID (Primary Key)

In [ ]:
# Escolher o arquivo dentro da bucket
bucket.blob('Person.Person.csv')

# Escolher uma variável que vai receber o caminho do arquivo que quero ler
path='gs://bucket-rox/Dados_Originais/Person.Person.csv'

In [ ]:
dfPerson = (spark
        .read
        .format ('csv')
        .option ('header','true')
        .option('inferschema','true')
        .option('delimiter',';')
        .load(path)
         )

In [ ]:
dfPerson.printSchema()

root
 |-- BusinessEntityID: integer (nullable = true)
 |-- PersonType: string (nullable = true)
 |-- NameStyle: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- MiddleName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- Suffix: string (nullable = true)
 |-- EmailPromotion: integer (nullable = true)
 |-- AdditionalContactInfo: string (nullable = true)
 |-- Demographics: string (nullable = true)
 |-- rowguid: string (nullable = true)
 |-- ModifiedDate: timestamp (nullable = true)



In [ ]:
# verificando as primeiras linhas
dfPerson.orderBy(col('BusinessEntityID').asc()).show(10,truncate=False)

+----------------+----------+---------+-----+---------+----------+----------+------+--------------+---------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------------------+
|BusinessEntityID|PersonType|NameStyle|Title|FirstName|MiddleName|LastName  |Suffix|EmailPromotion|AdditionalContactInfo|Demographics                                                                                                                                                           |rowguid                             |ModifiedDate       |
+----------------+----------+---------+-----+---------+----------+----------+------+--------------+---------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------

In [ ]:
# verificando as últimas linhas
dfPerson.orderBy(col('BusinessEntityID').asc()).tail(1)

[Row(BusinessEntityID=20777, PersonType='IN', NameStyle=0, Title='NULL', FirstName='Crystal', MiddleName='NULL', LastName='Hu', Suffix='NULL', EmailPromotion=0, AdditionalContactInfo='NULL', Demographics='"<IndividualSurvey xmlns=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/IndividualSurvey""><TotalPurchaseYTD>-19.5</TotalPurchaseYTD><DateFirstPurchase>2003-11-17Z</DateFirstPurchase><BirthDate>1971-05-05Z</BirthDate><MaritalStatus>M</MaritalStatus><YearlyIncome>75001-100000</YearlyIncome><Gender>F</Gender><TotalChildren>0</TotalChildren><NumberChildrenAtHome>0</NumberChildrenAtHome><Education>Bachelors </Education><Occupation>Professional</Occupation><HomeOwnerFlag>0</HomeOwnerFlag><NumberCarsOwned>4</NumberCarsOwned><CommuteDistance>10+ Miles</CommuteDistance></IndividualSurvey>"', rowguid='1773C8F3-2CC7-4826-A7F5-30D3F36543CD', ModifiedDate=datetime.datetime(2013, 10, 16, 0, 0))]

In [ ]:
# verificando se há registro null na coluna ID
dfPerson.createOrReplaceTempView("TAB")
spark.sql("SELECT BusinessEntityID FROM TAB WHERE BusinessEntityID is null ").show(truncate=False)

+----------------+
|BusinessEntityID|
+----------------+
+----------------+



In [ ]:
# Verificando se existe ID repetido
df1=dfPerson.groupBy("BusinessEntityID").count().filter("count > 1")
df1.drop('count').show()

+----------------+
|BusinessEntityID|
+----------------+
+----------------+



In [ ]:
# Verificando se existe linha repetida
df1=dfPerson.groupBy('BusinessEntityID','PersonType','NameStyle','Title','FirstName','MiddleName','LastName','Suffix','EmailPromotion','AdditionalContactInfo','Demographics','rowguid','ModifiedDate').count().filter("count > 1")
df1.drop('count').show()

+----------------+----------+---------+-----+---------+----------+--------+------+--------------+---------------------+------------+-------+------------+
|BusinessEntityID|PersonType|NameStyle|Title|FirstName|MiddleName|LastName|Suffix|EmailPromotion|AdditionalContactInfo|Demographics|rowguid|ModifiedDate|
+----------------+----------+---------+-----+---------+----------+--------+------+--------------+---------------------+------------+-------+------------+
+----------------+----------+---------+-----+---------+----------+--------+------+--------------+---------------------+------------+-------+------------+



In [ ]:
# necessário alterar "," para ";" porque estava gravando como mais uma coluna
dfPerson = dfPerson.withColumn('AdditionalContactInfo',regexp_replace('AdditionalContactInfo',',',';'))
dfPerson = dfPerson.withColumn('Demographics',regexp_replace('Demographics',',',';'))

In [ ]:
dfPerson.write.options(header=True,delimiter=',').csv('gs://bucket-rox/Dados_Tratados/Person')

# Tabela Production.Product: PoductID (Primary Key)

In [ ]:
# Escolher o arquivo dentro da bucket
bucket.blob('Production.Product.csv')

# Escolher uma variável que vai receber o caminho do arquivo que quero ler
path='gs://bucket-rox/Dados_Originais/Production.Product.csv'

In [ ]:
dfProduction = (spark
        .read
        .format ('csv')
        .option ('header','true')
        .option('inferschema','true')
        .option('delimiter',';')
        .load(path)
         )

In [ ]:
dfProduction.printSchema()

root
 |-- ProductID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- ProductNumber: string (nullable = true)
 |-- MakeFlag: integer (nullable = true)
 |-- FinishedGoodsFlag: integer (nullable = true)
 |-- Color: string (nullable = true)
 |-- SafetyStockLevel: integer (nullable = true)
 |-- ReorderPoint: integer (nullable = true)
 |-- StandardCost: string (nullable = true)
 |-- ListPrice: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- SizeUnitMeasureCode: string (nullable = true)
 |-- WeightUnitMeasureCode: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- DaysToManufacture: integer (nullable = true)
 |-- ProductLine: string (nullable = true)
 |-- Class: string (nullable = true)
 |-- Style: string (nullable = true)
 |-- ProductSubcategoryID: string (nullable = true)
 |-- ProductModelID: string (nullable = true)
 |-- SellStartDate: timestamp (nullable = true)
 |-- SellEndDate: string (nullable = true)
 |-- DiscontinuedDate: string (

In [ ]:
# verificando as primeiras linhas
dfProduction.orderBy(col('ProductID').asc()).show(10)

+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-----------+----------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|MakeFlag|FinishedGoodsFlag| Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|      SellStartDate|SellEndDate|DiscontinuedDate|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-

In [ ]:
# verificando as últimas linhas
dfProduction.orderBy(col('ProductID').asc()).tail(1)

[Row(ProductID=999, Name='Road-750 Black, 52', ProductNumber='BK-R19B-52', MakeFlag=1, FinishedGoodsFlag=1, Color='Black', SafetyStockLevel=100, ReorderPoint=75, StandardCost='343,6496', ListPrice='539,99', Size='52', SizeUnitMeasureCode='CM ', WeightUnitMeasureCode='LB ', Weight='20.42', DaysToManufacture=4, ProductLine='R ', Class='L ', Style='U ', ProductSubcategoryID='2', ProductModelID='31', SellStartDate=datetime.datetime(2013, 5, 30, 0, 0), SellEndDate='NULL', DiscontinuedDate='NULL', rowguid='AE638923-2B67-4679-B90E-ABBAB17DCA31', ModifiedDate=datetime.datetime(2014, 2, 8, 10, 1, 36, 827000))]

In [ ]:
# verificando se há registro null na coluna ID
dfProduction.createOrReplaceTempView("TAB")
spark.sql("SELECT ProductID FROM TAB WHERE ProductID is null ").show(truncate=False)

+---------+
|ProductID|
+---------+
+---------+



In [ ]:
# Verificando se existe ID repetido
df1=dfProduction.groupBy("ProductID").count().filter("count > 1")
df1.drop('count').show()

+---------+
|ProductID|
+---------+
+---------+



In [ ]:
# Verificando se existe linha repetida
df1=dfProduction.groupBy('ProductID','Name','ProductNumber','MakeFlag','FinishedGoodsFlag','Color','SafetyStockLevel','ReorderPoint','StandardCost','ListPrice','Size','SizeUnitMeasureCode','WeightUnitMeasureCode','Weight','DaysToManufacture','ProductLine','Class','Style','ProductSubcategoryID','ProductModelID','SellStartDate','SellEndDate','DiscontinuedDate','rowguid','ModifiedDate').count().filter("count > 1")
df1.drop('count').show()

+---------+----+-------------+--------+-----------------+-----+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------+-----------+----------------+-------+------------+
|ProductID|Name|ProductNumber|MakeFlag|FinishedGoodsFlag|Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|SellStartDate|SellEndDate|DiscontinuedDate|rowguid|ModifiedDate|
+---------+----+-------------+--------+-----------------+-----+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------+-----------+----------------+-------+------------+
+---------+----+-------------+--------+-----------------+----

In [ ]:
# gravando o arquivo tratado no bucket
dfProduction.write.options(header=True, delimiter=',').csv('gs://bucket-rox/Dados_Tratados/Production')

# Tabela Sales.SpecialOfferProduct: SpecialOfferID não pode ser Primary Key porque não é única, ProductID (Foreing Key)

Obs.:

Não existe uma Primary Key nesta tabela.

O ideal nesta caso é que haja uma revisão na sua modelagem pois, em teoria, uma tabela não deve ficar sem uma Primary Key.

Outra sugestão seria criar uma Primary Key da união das colunas ProductID e SpecialOfferID já que elas, juntas, não se repetem. Porém esta solução também precisa ser amplamente estudada para analisar o impacto sobre as demais tabelas.

In [ ]:
# Escolher o arquivo dentro da bucket
bucket.blob('Sales.SpecialOfferProduct.csv')

# Escolher uma variável que vai receber o caminho do arquivo que quero ler
path='gs://bucket-rox/Dados_Originais/Sales.SpecialOfferProduct.csv'

In [ ]:
dfSpecialOfferProduct = (spark
        .read
        .format ('csv')
        .option ('header','true') # para incluir o cabeçalho
        .option('inferschema','true') #para ele dizer o esquema
        .option('delimiter',';')
        .load(path)
         )

In [ ]:
dfSpecialOfferProduct.printSchema()

root
 |-- SpecialOfferID: integer (nullable = true)
 |-- ProductID: integer (nullable = true)
 |-- rowguid: string (nullable = true)
 |-- ModifiedDate: timestamp (nullable = true)



In [ ]:
# verificando as primeiras linhas
dfSpecialOfferProduct.orderBy(col('ProductID').asc(),col('SpecialOfferID').asc()).show(20)

+--------------+---------+--------------------+-------------------+
|SpecialOfferID|ProductID|             rowguid|       ModifiedDate|
+--------------+---------+--------------------+-------------------+
|             1|      680|BB30B868-D86C-455...|2011-04-01 00:00:00|
|             1|      706|B3C9A4B1-2AE6-4CB...|2011-04-01 00:00:00|
|             1|      707|27B711FE-0B77-4EA...|2011-04-01 00:00:00|
|             2|      707|CCA6DBA4-0483-429...|2011-05-01 00:00:00|
|             3|      707|5F32C413-EFD3-460...|2011-05-01 00:00:00|
|             8|      707|13302081-DE14-494...|2012-04-30 00:00:00|
|            11|      707|AF988F8E-4FEC-4C7...|2013-04-30 00:00:00|
|             1|      708|46CBB78B-246E-4D6...|2011-04-01 00:00:00|
|             2|      708|859A66B9-D4FA-44E...|2011-05-01 00:00:00|
|             3|      708|50C8F56C-ACEE-463...|2011-05-01 00:00:00|
|             8|      708|4C95EB32-7858-4DB...|2012-04-30 00:00:00|
|            11|      708|3754699E-F05F-451...|2

In [ ]:
# verificando as ultimas linhas
dfSpecialOfferProduct.orderBy(col('ProductID').asc()).tail(1)

[Row(SpecialOfferID=2, ProductID=999, rowguid='4FF26A13-4806-4B51-94D3-D545DAEC7825', ModifiedDate=datetime.datetime(2011, 5, 1, 0, 0))]

In [ ]:
dfSpecialOfferProduct.show()

+--------------+---------+--------------------+-------------------+
|SpecialOfferID|ProductID|             rowguid|       ModifiedDate|
+--------------+---------+--------------------+-------------------+
|             1|      680|BB30B868-D86C-455...|2011-04-01 00:00:00|
|             1|      706|B3C9A4B1-2AE6-4CB...|2011-04-01 00:00:00|
|             1|      707|27B711FE-0B77-4EA...|2011-04-01 00:00:00|
|             1|      708|46CBB78B-246E-4D6...|2011-04-01 00:00:00|
|             1|      709|CF102AA0-055F-4D2...|2011-04-01 00:00:00|
|             1|      710|63718DA1-464B-432...|2011-04-01 00:00:00|
|             1|      711|457EB971-D1C9-48C...|2011-04-01 00:00:00|
|             1|      712|5B948448-BAE5-4F2...|2011-04-01 00:00:00|
|             1|      713|07768F40-6E46-430...|2011-04-01 00:00:00|
|             1|      714|85004BCE-C74A-4D4...|2011-04-01 00:00:00|
|             1|      715|BE469616-B279-452...|2011-04-01 00:00:00|
|             1|      716|96D141FD-1D8E-431...|2

In [ ]:
# Verificando se existe ID repetido
df1=dfSpecialOfferProduct.groupBy("ProductID").count().filter("count > 1")
df1.drop('count').show()

+---------+
|ProductID|
+---------+
|      737|
|      858|
|      808|
|      883|
|      804|
|      970|
|      961|
|      974|
|      876|
|      976|
|      772|
|      874|
|      939|
|      762|
|      784|
|      988|
|      723|
|      844|
|      908|
|      806|
+---------+
only showing top 20 rows



In [ ]:
dfSpecialOfferProduct.filter(col('ProductID') == 737).show()

+--------------+---------+--------------------+-------------------+
|SpecialOfferID|ProductID|             rowguid|       ModifiedDate|
+--------------+---------+--------------------+-------------------+
|             1|      737|1F845481-CAA5-4F9...|2011-04-01 00:00:00|
|            12|      737|2D79385D-555C-403...|2013-04-30 00:00:00|
+--------------+---------+--------------------+-------------------+



In [ ]:
# Verificando se existem linhas repetidas
df1=dfSpecialOfferProduct.groupBy('SpecialOfferID','ProductID','rowguid','ModifiedDate').count().filter("count > 1")
df1.drop('count').show()

+--------------+---------+-------+------------+
|SpecialOfferID|ProductID|rowguid|ModifiedDate|
+--------------+---------+-------+------------+
+--------------+---------+-------+------------+



In [ ]:
dfSpecialOfferProduct.write.options(header=True, delimiter=',').csv('gs://bucket-rox/Dados_Tratados/SpecialOfferProduct')

# Tabela Sales.Customer: CustomerID (Primary Key)

In [ ]:
# Escolher o arquivo dentro da bucket
bucket.blob('Sales.Customer.csv')

# Escolher uma variável que vai receber o caminho do arquivo que quero ler
path='gs://bucket-rox/Dados_Originais/Sales.Customer.csv'

In [ ]:
dfSalesCustomer = (spark
        .read
        .format ('csv')
        .option ('header','true') # para incluir o cabeçalho
        .option('inferschema','true') #para ele dizer o esquema
        .option('delimiter',';')
        .load(path)
         )

In [ ]:
dfSalesCustomer.printSchema()

root
 |-- CustomerID: integer (nullable = true)
 |-- PersonID: string (nullable = true)
 |-- StoreID: string (nullable = true)
 |-- TerritoryID: integer (nullable = true)
 |-- AccountNumber: string (nullable = true)
 |-- rowguid: string (nullable = true)
 |-- ModifiedDate: timestamp (nullable = true)



In [ ]:
# verificando as primeiras linhas
dfSalesCustomer.orderBy(col('CustomerID').asc(),col('PersonID').asc()).show(20)

+----------+--------+-------+-----------+-------------+--------------------+--------------------+
|CustomerID|PersonID|StoreID|TerritoryID|AccountNumber|             rowguid|        ModifiedDate|
+----------+--------+-------+-----------+-------------+--------------------+--------------------+
|         1|    NULL|    934|          1|   AW00000001|3F5AE95E-B87D-4AE...|2014-09-12 11:15:...|
|         2|    NULL|   1028|          1|   AW00000002|E552F657-A9AF-4A7...|2014-09-12 11:15:...|
|         3|    NULL|    642|          4|   AW00000003|130774B1-DB21-4EF...|2014-09-12 11:15:...|
|         4|    NULL|    932|          4|   AW00000004|FF862851-1DAA-404...|2014-09-12 11:15:...|
|         5|    NULL|   1026|          4|   AW00000005|83905BDC-6F5E-4F7...|2014-09-12 11:15:...|
|         6|    NULL|    644|          4|   AW00000006|1A92DF88-BFA2-467...|2014-09-12 11:15:...|
|         7|    NULL|    930|          1|   AW00000007|03E9273E-B193-448...|2014-09-12 11:15:...|
|         8|    NULL

In [ ]:
dfSalesCustomer = dfSalesCustomer.withColumn('PersonID',dfSalesCustomer['PersonID'].cast(IntegerType()))

In [ ]:
dfSalesCustomer.printSchema()

root
 |-- CustomerID: integer (nullable = true)
 |-- PersonID: integer (nullable = true)
 |-- StoreID: string (nullable = true)
 |-- TerritoryID: integer (nullable = true)
 |-- AccountNumber: string (nullable = true)
 |-- rowguid: string (nullable = true)
 |-- ModifiedDate: timestamp (nullable = true)



In [ ]:
dfSalesCustomer.show()

+----------+--------+-------+-----------+-------------+--------------------+--------------------+
|CustomerID|PersonID|StoreID|TerritoryID|AccountNumber|             rowguid|        ModifiedDate|
+----------+--------+-------+-----------+-------------+--------------------+--------------------+
|         1|    null|    934|          1|   AW00000001|3F5AE95E-B87D-4AE...|2014-09-12 11:15:...|
|         2|    null|   1028|          1|   AW00000002|E552F657-A9AF-4A7...|2014-09-12 11:15:...|
|         3|    null|    642|          4|   AW00000003|130774B1-DB21-4EF...|2014-09-12 11:15:...|
|         4|    null|    932|          4|   AW00000004|FF862851-1DAA-404...|2014-09-12 11:15:...|
|         5|    null|   1026|          4|   AW00000005|83905BDC-6F5E-4F7...|2014-09-12 11:15:...|
|         6|    null|    644|          4|   AW00000006|1A92DF88-BFA2-467...|2014-09-12 11:15:...|
|         7|    null|    930|          1|   AW00000007|03E9273E-B193-448...|2014-09-12 11:15:...|
|         8|    null

In [ ]:
# verificando as últimas linhas
dfSalesCustomer.orderBy(col('CustomerID').asc(),col('PersonID').asc()).tail(1)

[Row(CustomerID=30118, PersonID=1993, StoreID='1994', TerritoryID=3, AccountNumber='AW00030118', rowguid='2495B4EB-FE8B-459E-A1B6-DBA25C04E626', ModifiedDate=datetime.datetime(2014, 9, 12, 11, 15, 7, 263000))]

In [ ]:
# verificando se há registro null na coluna ID
dfSalesCustomer.createOrReplaceTempView("TAB")
spark.sql("SELECT CustomerID FROM TAB WHERE CustomerID is null ").show(truncate=False)

+----------+
|CustomerID|
+----------+
+----------+



In [ ]:
# Verificando se existe ID Cliente repetido
df1=dfSalesCustomer.groupBy("CustomerID").count().filter("count > 1")
df1.drop('count').show()

+----------+
|CustomerID|
+----------+
+----------+



In [ ]:
# Verificando se existe ID repetido
df1=dfSalesCustomer.groupBy("PersonID").count().filter("count > 1")
df1.drop('count').show()

+--------+
|PersonID|
+--------+
|    null|
+--------+



In [ ]:
# Verificando se existe ID repetido
df1=dfSalesCustomer.groupBy("StoreID").count().filter("count > 1")
df1.drop('count').show()

+-------+
|StoreID|
+-------+
|   1436|
|    296|
|   1090|
|    800|
|    944|
|   1372|
|   1394|
|   1280|
|   1870|
|    926|
|    870|
|    666|
|   1808|
|    718|
|    574|
|   1880|
|    544|
|    740|
|    334|
|    442|
+-------+
only showing top 20 rows



In [ ]:
# Verificando se existe linha repetida
df1=dfSalesCustomer.groupBy('CustomerID','PersonID','StoreID','TerritoryID','AccountNumber','rowguid','ModifiedDate').count().filter("count > 1")
df1.drop('count').show()

+----------+--------+-------+-----------+-------------+-------+------------+
|CustomerID|PersonID|StoreID|TerritoryID|AccountNumber|rowguid|ModifiedDate|
+----------+--------+-------+-----------+-------------+-------+------------+
+----------+--------+-------+-----------+-------------+-------+------------+



In [ ]:
dfSalesCustomer.write.options(header=True, delimiter=',').csv('gs://bucket-rox/Dados_Tratados/SalesCustomer')

# Tabela Sales.SalesOrderDetail: SalesOrderDetailID (Primary Key), ProductID e SalesOrderID (Foreing Key)

In [ ]:
# Escolher o arquivo dentro da bucket
bucket.blob('Sales.SalesOrderDetail.csv')

# Escolher uma variável que vai receber o caminho do arquivo que quero ler
path='gs://bucket-rox/Dados_Originais/Sales.SalesOrderDetail.csv'

In [ ]:
dfSalesOrderDetail = (spark
        .read
        .format ('csv')
        .option ('header','true') # para incluir o cabeçalho
        .option('inferschema','true') #para ele dizer o esquema
        .option('delimiter',';')
        .load(path)
         )

In [ ]:
dfSalesOrderDetail.printSchema()

root
 |-- SalesOrderID: integer (nullable = true)
 |-- SalesOrderDetailID: integer (nullable = true)
 |-- CarrierTrackingNumber: string (nullable = true)
 |-- OrderQty: integer (nullable = true)
 |-- ProductID: integer (nullable = true)
 |-- SpecialOfferID: integer (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- UnitPriceDiscount: string (nullable = true)
 |-- LineTotal: double (nullable = true)
 |-- rowguid: string (nullable = true)
 |-- ModifiedDate: timestamp (nullable = true)



In [ ]:
# verificando as primeiras linhas
dfSalesOrderDetail.orderBy(col('SalesOrderDetailID').asc()).show(20)

+------------+------------------+---------------------+--------+---------+--------------+---------+-----------------+---------+--------------------+-------------------+
|SalesOrderID|SalesOrderDetailID|CarrierTrackingNumber|OrderQty|ProductID|SpecialOfferID|UnitPrice|UnitPriceDiscount|LineTotal|             rowguid|       ModifiedDate|
+------------+------------------+---------------------+--------+---------+--------------+---------+-----------------+---------+--------------------+-------------------+
|       43659|                 1|         4911-403C-98|       1|      776|             1| 2024,994|             0,00| 2024.994|B207C96D-D9E6-402...|2011-05-31 00:00:00|
|       43659|                 2|         4911-403C-98|       3|      777|             1| 2024,994|             0,00| 6074.982|7ABB600D-1E77-41B...|2011-05-31 00:00:00|
|       43659|                 3|         4911-403C-98|       1|      778|             1| 2024,994|             0,00| 2024.994|475CF8C6-49F6-486...|2011-05

In [ ]:
# verificando as últimas linhas
dfSalesOrderDetail.orderBy(col('SalesOrderDetailID').asc()).tail(1)

[Row(SalesOrderID=75123, SalesOrderDetailID=121317, CarrierTrackingNumber='NULL', OrderQty=1, ProductID=712, SpecialOfferID=1, UnitPrice='8,99', UnitPriceDiscount='0,00', LineTotal=8.99, rowguid='73646D26-0461-450D-8019-2C6C858619CA', ModifiedDate=datetime.datetime(2014, 6, 30, 0, 0))]

In [ ]:
# verificando se há registro null na coluna ID
dfSalesOrderDetail.createOrReplaceTempView("TAB")
spark.sql("SELECT SalesOrderDetailID FROM TAB WHERE SalesOrderDetailID is null ").show(truncate=False)

+------------------+
|SalesOrderDetailID|
+------------------+
+------------------+



In [ ]:
# Verificando se existe ID repetido
df1=dfSalesOrderDetail.groupBy("SalesOrderDetailID").count().filter("count > 1")
df1.drop('count').show()

+------------------+
|SalesOrderDetailID|
+------------------+
+------------------+



In [ ]:
# Verificando se existe linha repetida
df1=dfSalesOrderDetail.groupBy('SalesOrderID','SalesOrderDetailID','CarrierTrackingNumber','OrderQty','ProductID','SpecialOfferID','UnitPrice','UnitPriceDiscount','LineTotal','rowguid','ModifiedDate').count().filter("count > 1")
df1.drop('count').show()

+------------+------------------+---------------------+--------+---------+--------------+---------+-----------------+---------+-------+------------+
|SalesOrderID|SalesOrderDetailID|CarrierTrackingNumber|OrderQty|ProductID|SpecialOfferID|UnitPrice|UnitPriceDiscount|LineTotal|rowguid|ModifiedDate|
+------------+------------------+---------------------+--------+---------+--------------+---------+-----------------+---------+-------+------------+
+------------+------------------+---------------------+--------+---------+--------------+---------+-----------------+---------+-------+------------+



In [ ]:
dfSalesOrderDetail.write.options(header=True, delimiter=',').csv('gs://bucket-rox/Dados_Tratados/SalesOrderDetail')

# Tabela Sales.SalesOrderHeader: SalesOrderID (Primary Key), CustomerID (Foreing Key)

In [ ]:
# Escolher o arquivo dentro da bucket
bucket.blob('Sales.SalesOrderHeader.csv')

# Escolher uma variável que vai receber o caminho do arquivo que quero ler
path='gs://bucket-rox/Dados_Originais/Sales.SalesOrderHeader.csv'

In [ ]:
dfSalesOrderHeader = (spark
        .read
        .format ('csv')
        .option ('header','true') # para incluir o cabeçalho
        .option('inferschema','true') #para ele dizer o esquema
        .option('delimiter',';')
        .load(path)
         )

In [ ]:
dfSalesOrderHeader.printSchema()

root
 |-- SalesOrderID: integer (nullable = true)
 |-- RevisionNumber: integer (nullable = true)
 |-- OrderDate: timestamp (nullable = true)
 |-- DueDate: timestamp (nullable = true)
 |-- ShipDate: timestamp (nullable = true)
 |-- Status: integer (nullable = true)
 |-- OnlineOrderFlag: integer (nullable = true)
 |-- SalesOrderNumber: string (nullable = true)
 |-- PurchaseOrderNumber: string (nullable = true)
 |-- AccountNumber: string (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- SalesPersonID: string (nullable = true)
 |-- TerritoryID: integer (nullable = true)
 |-- BillToAddressID: integer (nullable = true)
 |-- ShipToAddressID: integer (nullable = true)
 |-- ShipMethodID: integer (nullable = true)
 |-- CreditCardID: string (nullable = true)
 |-- CreditCardApprovalCode: string (nullable = true)
 |-- CurrencyRateID: string (nullable = true)
 |-- SubTotal: string (nullable = true)
 |-- TaxAmt: string (nullable = true)
 |-- Freight: string (nullable = true)
 |-- Tota

In [ ]:
# verificando a coluna DateOrder identifiquei que o time não tem valores
df2=dfSalesOrderHeader.withColumn('Date',F.substring(F.col('OrderDate'),1,10)).withColumn('Time',F.substring(F.col('OrderDate'),12,19))

In [ ]:
df2.orderBy(col('Time').desc()).show(20)

+------------+--------------+-------------------+-------------------+-------------------+------+---------------+----------------+-------------------+--------------+----------+-------------+-----------+---------------+---------------+------------+------------+----------------------+--------------+----------+---------+---------+----------+-------+--------------------+-------------------+----------+--------+
|SalesOrderID|RevisionNumber|          OrderDate|            DueDate|           ShipDate|Status|OnlineOrderFlag|SalesOrderNumber|PurchaseOrderNumber| AccountNumber|CustomerID|SalesPersonID|TerritoryID|BillToAddressID|ShipToAddressID|ShipMethodID|CreditCardID|CreditCardApprovalCode|CurrencyRateID|  SubTotal|   TaxAmt|  Freight|  TotalDue|Comment|             rowguid|       ModifiedDate|      Date|    Time|
+------------+--------------+-------------------+-------------------+-------------------+------+---------------+----------------+-------------------+--------------+----------+-------

In [ ]:
dfSalesOrderHeader=dfSalesOrderHeader.withColumn('OrderDate',F.substring(F.col('OrderDate'),1,10))

In [ ]:
dfSalesOrderHeader.show()

+------------+--------------+----------+-------------------+-------------------+------+---------------+----------------+-------------------+--------------+----------+-------------+-----------+---------------+---------------+------------+------------+----------------------+--------------+----------+---------+---------+----------+-------+--------------------+-------------------+
|SalesOrderID|RevisionNumber| OrderDate|            DueDate|           ShipDate|Status|OnlineOrderFlag|SalesOrderNumber|PurchaseOrderNumber| AccountNumber|CustomerID|SalesPersonID|TerritoryID|BillToAddressID|ShipToAddressID|ShipMethodID|CreditCardID|CreditCardApprovalCode|CurrencyRateID|  SubTotal|   TaxAmt|  Freight|  TotalDue|Comment|             rowguid|       ModifiedDate|
+------------+--------------+----------+-------------------+-------------------+------+---------------+----------------+-------------------+--------------+----------+-------------+-----------+---------------+---------------+------------+---

In [ ]:
# verificando as primeiras linhas
dfSalesOrderHeader.orderBy(col('SalesOrderID').asc()).show(20)

+------------+--------------+-------------------+-------------------+-------------------+------+---------------+----------------+-------------------+--------------+----------+-------------+-----------+---------------+---------------+------------+------------+----------------------+--------------+----------+---------+---------+----------+-------+--------------------+-------------------+
|SalesOrderID|RevisionNumber|          OrderDate|            DueDate|           ShipDate|Status|OnlineOrderFlag|SalesOrderNumber|PurchaseOrderNumber| AccountNumber|CustomerID|SalesPersonID|TerritoryID|BillToAddressID|ShipToAddressID|ShipMethodID|CreditCardID|CreditCardApprovalCode|CurrencyRateID|  SubTotal|   TaxAmt|  Freight|  TotalDue|Comment|             rowguid|       ModifiedDate|
+------------+--------------+-------------------+-------------------+-------------------+------+---------------+----------------+-------------------+--------------+----------+-------------+-----------+---------------+-----

In [ ]:
# verificando as últimas linhas
dfSalesOrderHeader.orderBy(col('SalesOrderID').asc()).tail(1)

[Row(SalesOrderID=75123, RevisionNumber=8, OrderDate=datetime.datetime(2014, 6, 30, 0, 0), DueDate=datetime.datetime(2014, 7, 12, 0, 0), ShipDate=datetime.datetime(2014, 7, 7, 0, 0), Status=5, OnlineOrderFlag=1, SalesOrderNumber='SO75123', PurchaseOrderNumber='NULL', AccountNumber='10-4030-018759', CustomerID=18759, SalesPersonID='NULL', TerritoryID=6, BillToAddressID=14024, ShipToAddressID=14024, ShipMethodID=1, CreditCardID='10084', CreditCardApprovalCode='230370Vi51970', CurrencyRateID='NULL', SubTotal='189,97', TaxAmt='15,1976', Freight='4,7493', TotalDue='209,9169', Comment='NULL', rowguid='D54752FF-2B54-4BE5-95EA-3B72289C059F', ModifiedDate=datetime.datetime(2014, 7, 7, 0, 0))]

In [ ]:
# verificando se há registro null na coluna ID
dfSalesOrderHeader.createOrReplaceTempView("TAB")
spark.sql("SELECT SalesOrderID FROM TAB WHERE SalesOrderID is null ").show(truncate=False)

+------------+
|SalesOrderID|
+------------+
+------------+



In [ ]:
# Verificando se existe ID repetido
df1=dfSalesOrderHeader.groupBy("SalesOrderID").count().filter("count > 1")
df1.drop('count').show()

+------------+
|SalesOrderID|
+------------+
+------------+



In [ ]:
# Verificando se existe ID repetido
df1=dfSalesOrderHeader.groupBy("CustomerID").count().filter("count > 1")
df1.drop('count').show()

+----------+
|CustomerID|
+----------+
|     11033|
|     29834|
|     28664|
|     14832|
|     22373|
|     11317|
|     15619|
|     12027|
|     29744|
|     29601|
|     29993|
|     13840|
|     29719|
|     26623|
|     26708|
|     17753|
|     23571|
|     24171|
|     27484|
|     15790|
+----------+
only showing top 20 rows



In [ ]:
dfSalesOrderHeader.filter(col('CustomerID') == 11033).show()

+------------+--------------+-------------------+-------------------+-------------------+------+---------------+----------------+-------------------+--------------+----------+-------------+-----------+---------------+---------------+------------+------------+----------------------+--------------+--------+--------+-------+---------+-------+--------------------+-------------------+
|SalesOrderID|RevisionNumber|          OrderDate|            DueDate|           ShipDate|Status|OnlineOrderFlag|SalesOrderNumber|PurchaseOrderNumber| AccountNumber|CustomerID|SalesPersonID|TerritoryID|BillToAddressID|ShipToAddressID|ShipMethodID|CreditCardID|CreditCardApprovalCode|CurrencyRateID|SubTotal|  TaxAmt|Freight| TotalDue|Comment|             rowguid|       ModifiedDate|
+------------+--------------+-------------------+-------------------+-------------------+------+---------------+----------------+-------------------+--------------+----------+-------------+-----------+---------------+---------------+-

In [ ]:
# Verificando se existe linha repetida
df1=dfSalesOrderHeader.groupBy('SalesOrderID','RevisionNumber','OrderDate','DueDate','ShipDate','Status','OnlineOrderFlag','SalesOrderNumber','PurchaseOrderNumber','AccountNumber','CustomerID','SalesPersonID','TerritoryID','BillToAddressID','ShipToAddressID','ShipMethodID','CreditCardID','CreditCardApprovalCode','CurrencyRateID','SubTotal','TaxAmt','Freight','TotalDue','Comment','rowguid','ModifiedDate').count().filter("count > 1")
df1.drop('count').show()

+------------+--------------+---------+-------+--------+------+---------------+----------------+-------------------+-------------+----------+-------------+-----------+---------------+---------------+------------+------------+----------------------+--------------+--------+------+-------+--------+-------+-------+------------+
|SalesOrderID|RevisionNumber|OrderDate|DueDate|ShipDate|Status|OnlineOrderFlag|SalesOrderNumber|PurchaseOrderNumber|AccountNumber|CustomerID|SalesPersonID|TerritoryID|BillToAddressID|ShipToAddressID|ShipMethodID|CreditCardID|CreditCardApprovalCode|CurrencyRateID|SubTotal|TaxAmt|Freight|TotalDue|Comment|rowguid|ModifiedDate|
+------------+--------------+---------+-------+--------+------+---------------+----------------+-------------------+-------------+----------+-------------+-----------+---------------+---------------+------------+------------+----------------------+--------------+--------+------+-------+--------+-------+-------+------------+
+------------+--------

In [ ]:
dfSalesOrderHeader.write.options(header=True, delimiter=',').csv('gs://bucket-rox/Dados_Tratados/SalesOrderHeader')